# Create a PubMed Research Agent with Strands Agents

In [ ]:
%pip install -U boto3 strands-agents strands-agents-tools

In [ ]:
import logging

# Enables Strands debug log level
logging.getLogger("strands").setLevel(logging.INFO)

# Sets the logging format and streams logs to stderr
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", handlers=[logging.StreamHandler()]
)

In [ ]:
QUERY = "What are some recent advances in GLP-1 drugs?"

## Basic Prompt without Context

In [ ]:
from strands import Agent

SYSTEM_PROMPT = """
    You are a specialized PubMed research agent. Your role is to:
    1. Search PubMed for medical papers related to the query
    2. Extract and summarize the most relevant clinical findings
    3. Identify key research groups and methodologies
    4. Return structured, well-cited information with PMCID references
    """

# Initialize your agent
agent = Agent(system_prompt=SYSTEM_PROMPT, load_tools_from_directory=False)

# Send a message to the agent
response = agent(QUERY)

Copy and paste a few of the PMCIDs in the previous cell into the [PubMed web search](https://pubmed.ncbi.nlm.nih.gov/). Notice anything unusual? They likely point to completely unrelated resources! Without additional context, LLMs will do their best to generate IDs that seem convincing - they may even return real IDs included in their training data. However, if we want our agent to consistently return accurate, up-to-date results we need to provide it with a tool.

## Search PubMed for Abstract Context

In [ ]:
from strands import Agent
from tools import search_pubmed

SYSTEM_PROMPT = """You are a life science research assistant. When given a scientific question, follow this process:

1. Use the search_pubmed tool with rerank="referenced_by", max_results to 200-500, and max_records to 20-50 to find highly-cited papers. Search broadly first, then narrow down. Use temporal filters like "last 5 years"[dp] for recent work. 
2. Extract and summarize the most relevant clinical findings.
3. Return structured, well-cited information with PMID references.

Key guidelines:
- Always use rerank="referenced_by" in searches to prioritize influential papers.
- Limit searches to 20-50 articles for focused analysis.
- Select articles strategically based on citation count and relevance.
"""

# Initialize your agent
agent = Agent(
    system_prompt=SYSTEM_PROMPT, tools=[search_pubmed], load_tools_from_directory=False
)

# Send a message to the agent
response = agent(QUERY)

## Retrieve Full Text

In [ ]:
from tools import search_pubmed, read_pubmed

SYSTEM_PROMPT = """You are a life science research assistant. When given a scientific question, follow this process:

1. Use the search_pubmed tool with rerank="referenced_by", max_results to 200-500, and max_records to 20-50 to find highly-cited papers. Search broadly first, then narrow down. Use temporal filters like "last 5 years"[dp] for recent work. 
2. Use read_pubmed on the 1-2 most relevant articles from your search results to gain a better understanding of the space. Focus on highly-cited papers and reviews.
3. Extract and summarize the most relevant clinical findings.
3. Return structured, well-cited information with PMID references.

Key guidelines:
- Always use rerank="referenced_by" in searches to prioritize influential papers.
- Limit searches to 20-50 articles for focused analysis.
- Select articles strategically based on citation count and relevance.
"""

# Initialize your agent
agent = Agent(
    system_prompt=SYSTEM_PROMPT,
    tools=[search_pubmed, read_pubmed],
    load_tools_from_directory=False,
)

# Send a message to the agent
response = agent(QUERY)